In [17]:
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'model.pt')

In [1]:
import torch
model = torch.jit.load('model.pt')

In [1]:
from utils import *

def test():
    # Load model

    # load model from s3
    # s3 = boto3.client('s3')
    # bucket_name = 'your_bucket_name'
    # object_key = 'your_object_key'

    # response = s3.get_object(Bucket=bucket_name, Key=object_key)
    # data = response['Body'].read()

    # save model to local
    # with open('/tmp/model.pt', 'wb') as file:
    #     file.write(data)

    # load model from local
    # model = torch.jit.load('/tmp/model.pt')
    model = torch.jit.load('model.pt')
    
    # import input from database
    r_seq = db2Rseq()
    h_seq = db2Hseq(h_step = 4)
    t = dt2T()
    s_attrs, _ = db2S()

    # model inference
    minutes = [20, 40, 60, 120]

    for i, m in enumerate(minutes):
        globals()[f'res_{m}'] = model(r_seq.float(), h_seq[:, i, :, :].float(), t[:, i, :].int(), s_attrs.float())

    # export output to database
    now = db2Now()
    outputs = torch.hstack([now, res_20, res_40, res_60, res_120])

    try:
        y2db(outputs) 
        return {'status':'Success', 'time': f'{get_now()}'}

    except: return {'status':'Failed'}

In [2]:
test()

{'status': 'Success', 'time': '2023-08-24 10:20:00'}

In [3]:
from utils import *

model = torch.jit.load('model.pt')
    
# import input from database
r_seq = db2Rseq()
h_seq = db2Hseq(h_step = 4)
t = dt2T()
s_attrs, _ = db2S()

# model inference
minutes = [20, 40, 60, 120]

for i, m in enumerate(minutes):
    globals()[f'res_{m}'] = model(r_seq.float(), h_seq[:, i, :, :].float(), t[:, i, :].int(), s_attrs.float())

# export output to database
now = db2Now()
outputs = torch.hstack([now, res_20, res_40, res_60, res_120])

In [4]:
outputs

tensor([[ 0.0000e+00, -5.3852e-03, -5.6336e+00,  ..., -5.3852e-03,
         -5.6336e+00, -6.3228e+00],
        [ 2.0000e+00, -7.9920e-01, -1.8177e+00,  ..., -7.9920e-01,
         -1.8177e+00, -9.4698e-01],
        [ 2.0000e+00, -8.3606e-01, -1.8029e+00,  ..., -8.3606e-01,
         -1.8029e+00, -9.1189e-01],
        ...,
        [ 0.0000e+00, -5.1188e-03, -5.6792e+00,  ..., -5.1188e-03,
         -5.6792e+00, -6.3833e+00],
        [ 0.0000e+00, -5.1188e-03, -5.6792e+00,  ..., -5.1188e-03,
         -5.6792e+00, -6.3833e+00],
        [ 0.0000e+00, -5.1188e-03, -5.6792e+00,  ..., -5.1188e-03,
         -5.6792e+00, -6.3833e+00]], dtype=torch.float64,
       grad_fn=<CatBackward0>)